In [78]:
import numpy as np
import pandas as pd
import random
from collections import Counter

from scipy.sparse import csr_matrix
import scipy.sparse

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score, reciprocal_rank, recall_at_k
from lightfm.cross_validation import random_train_test_split

In [27]:
# load in data
r = pd.read_csv("review_r_and_f", index_col="review_id")
print(r.shape)

b = pd.read_csv("business_r_and_f_8.4.csv", index_col="business_id")
#b = pd.read_csv("business_r_and_f.csv", index_col="business_id")
print(b.shape)

u = pd.read_csv("user_r_and_f", index_col="user_id")
print(u.shape)

(1953249, 8)
(18504, 202)
(654161, 22)


In [295]:
b2 = pd.read_csv("business_partial.csv", index_col="business_id")
b2

,name,city,stars,price,review_count,1star_counts,2star_counts,3star_counts,4star_counts,5star_counts,...,longitude,monday,tuesday,wednesday,thursday,friday,saturday,sunday,las_vegas,word_count
business_id,,,,,,,,,,,,,,,,,,,,,
--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,Las Vegas,4.0,4.0,1546,51.0,137.0,176.0,420.0,762.0,...,-115.169190,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,1,145.625485
--DaPTJW3-tB1vP-PfdTEg,Sunnyside Grill,Toronto,3.5,2.0,45,4.0,3.0,9.0,16.0,13.0,...,-79.444674,7:0-15:0,7:0-15:0,7:0-15:0,7:0-15:0,7:0-16:0,7:0-16:0,7:0-16:0,0,85.466667
--DdmeR16TRb3LsjG0ejrQ,World Food Championships,Las Vegas,3.0,NaN,5,0.0,2.0,0.0,3.0,0.0,...,-115.170975,0,0,0,0,10:0-22:0,10:0-22:0,10:0-19:0,1,433.000000
--KCl2FvVQpvjzmZSPyviA,Hungry Howie's Pizza,Charlotte,3.0,1.0,13,3.0,2.0,3.0,1.0,4.0,...,-80.796171,10:30-0:0,10:30-0:0,10:30-0:0,10:30-0:0,10:30-1:0,10:30-1:0,10:30-0:0,0,89.846154
--cZ6Hhc9F7VkKXxHMVZSQ,Pio Pio,Charlotte,4.0,2.0,333,22.0,22.0,43.0,111.0,135.0,...,-80.844820,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,12:0-21:0,0,99.252252
--o5BoU7qYMALeVDK6mwVg,Nightlife Tours,Las Vegas,3.5,NaN,6,2.0,0.0,0.0,1.0,3.0,...,-115.131625,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,1,283.666667
--orEUqwTzz5QKbmyYbAWw,Advantage Rent-A-Car,Phoenix,2.5,NaN,3,2.0,0.0,0.0,0.0,1.0,...,-112.047266,0,0,0,0,0,0,0,0,48.333333
-01XupAWZEXbdNbxNg5mEg,18 Degrees Neighborhood Grill,Scottsdale,3.0,2.0,82,20.0,14.0,17.0,22.0,9.0,...,-111.881925,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,0,122.329268
-03HVYxkeYWaafEpNJo1SA,GoodLife Vapes,Toronto,4.5,2.0,6,1.0,0.0,0.0,0.0,5.0,...,-79.397546,10:0-20:0,10:0-20:0,10:0-20:0,10:0-20:0,10:0-20:0,10:0-20:0,12:0-18:0,0,181.166667


In [296]:
[col for col in b2.columns]

['name',
 'city',
 'stars',
 'price',
 'review_count',
 '1star_counts',
 '2star_counts',
 '3star_counts',
 '4star_counts',
 '5star_counts',
 'text',
 'text_friendly',
 'text_reasonable',
 'text_local',
 'text_comfortable',
 'text_easy',
 'text_quick',
 'categories',
 'categories_clean',
 'cat_Restaurants',
 'cat_Food',
 'cat_Shopping',
 'cat_Active Life',
 'cat_Arts & Entertainment',
 'cat_Nightlife',
 'cat_Hotels & Travel',
 'cat_Tours',
 'cat_Landmark',
 'cat_Others',
 'state',
 'address',
 'neighborhood',
 'postal_code',
 'latitude',
 'longitude',
 'monday',
 'tuesday',
 'wednesday',
 'thursday',
 'friday',
 'saturday',
 'sunday',
 'las_vegas',
 'word_count']

In [28]:
b.head()

,name,city,las_vegas,stars,price,review_count,1star_ratio,2star_ratio,3star_ratio,4star_ratio,...,Thai,Turkish,Vegan,Vegetarian,Venues & Event Spaces,Vietnamese,Waffles,Wine & Spirits,Wine Bars,Wraps
business_id,,,,,,,,,,,,,,,,,,,,,
--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,Las Vegas,1,4.0,4.0,7.344073,0.032988,0.088616,0.113842,0.271669,...,0,0,0,0,0,0,0,0,0,0
--DaPTJW3-tB1vP-PfdTEg,Sunnyside Grill,Toronto,0,3.5,2.0,3.828641,0.088889,0.066667,0.200000,0.355556,...,0,0,0,0,0,0,0,0,0,0
--KCl2FvVQpvjzmZSPyviA,Hungry Howie's Pizza,Charlotte,0,3.0,1.0,2.639057,0.230769,0.153846,0.230769,0.076923,...,0,0,0,0,0,0,0,0,0,0
--cZ6Hhc9F7VkKXxHMVZSQ,Pio Pio,Charlotte,0,4.0,2.0,5.811141,0.066066,0.066066,0.129129,0.333333,...,0,0,0,0,0,0,0,0,0,0
-01XupAWZEXbdNbxNg5mEg,18 Degrees Neighborhood Grill,Scottsdale,0,3.0,2.0,4.418841,0.243902,0.170732,0.207317,0.268293,...,0,0,0,0,0,0,0,0,0,0


In [29]:
r.head()

,business_id,cool,date,funny,stars,text,useful,user_id
review_id,,,,,,,,
x7mDIiDB3jEiPGPHOmDzyw,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
dDl8zu1vWPdKGihJrwQbpw,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g
LZp4UX5zK3e-c5ZGSeo3kA,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,1,Terrible. Dry corn bread. Rib tips were all fa...,3,msQe1u7Z_XuqjGoqhB0J5g
Er4NBWCmCD4nM8_p1GRdow,elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,2,Back in 2005-2007 this place was my FAVORITE t...,2,msQe1u7Z_XuqjGoqhB0J5g
kUZWBVZvhWuC8TWUg5AYyA,zdE82PiD6wquvjYLyhOJNA,0,2012-04-18,1,5,AMAZING!!!\n\n I was referred here by a friend...,0,msQe1u7Z_XuqjGoqhB0J5g


In [30]:
u.head()

,name,review_count,average_stars,yelping_since,friends,fans,elite,cool,funny,useful,...,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,friend_count
user_id,,,,,,,,,,,,,,,,,,,,,
lzlZwIpuSWXEnNS91wxjHw,Susan,1,2.00,2015-09-28,None,0,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
XvLBr-9smbI0m_a7dXtB7w,Daipayan,2,5.00,2015-09-05,None,0,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
QPT4Ud4H5sJVr68yXhoWFw,Andy,1,4.00,2016-07-21,None,0,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
s4FoIXE_LSGviTHBe8dmcg,Shashank,3,3.00,2017-06-18,None,0,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
nnB0AE1Cxp_0154xkhXelw,Mike,11,2.77,2014-07-11,None,1,None,0,0,8,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# convert stars values into positive or negative
r['sentiment'] = r['stars'].apply(lambda x: 1 if x >3 else -1)

### Subset to Las Vegas and Phoenix Only

In [32]:
b = b[b.city.isin(['Las Vegas', 'Phoenix'])]
print(b.shape)

r = r[r.business_id.isin(b.index)]
print(r.shape)

u = u[u.index.isin(r.user_id)]
print(u.shape)

(8329, 202)
(1291335, 9)
(482806, 22)


In [83]:
# examine interactions per user

r.shape[0]/u.shape[0]

2.6746457169132114

### Las Vegas-Phoenix Intersection

In [34]:
r['city'] = r['business_id'].map(b['city'])
r_pos['city'] = r['business_id'].map(b['city'])

C:\Users\mm4me\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [113]:
# users in Las Vegas
u_lv = Counter([u for u in r['user_id'][r['city'] == 'Las Vegas']])

# users in Phoenix
u_ph = Counter([u for u in r['user_id'][r['city'] == 'Phoenix']])

In [114]:
# get intersection of users in Las Vegas and Phoenix
u_lv_ph = list(set([u for u in u_lv.keys()]) & set([u for u in u_ph.keys()]))
len(u_lv_ph)

13346

In [115]:
# subset by users in both cities and businesses that have been rated by those users
r_lv_ph = r[(r['user_id'].isin(u_lv_ph)) & (r['city'].str.contains('Las Vegas|Phoenix'))]
print(r_lv_ph.business_id.nunique(), r_lv_ph.user_id.nunique())

6996 13346


### Subset business and user features

In [292]:
[col for col in b.columns]

['name',
 'city',
 'las_vegas',
 'stars',
 'price',
 'review_count',
 '1star_ratio',
 '2star_ratio',
 '3star_ratio',
 '4star_ratio',
 '5star_ratio',
 'text',
 'word_count',
 'text_friendly',
 'text_reasonable',
 'text_local',
 'text_comfortable',
 'text_easy',
 'text_quick',
 'cat_Restaurants',
 'cat_Food',
 'cat_Shopping',
 'cat_Active Life',
 'cat_Arts & Entertainment',
 'cat_Nightlife',
 'cat_Hotels & Travel',
 'cat_Tours',
 'state',
 'address',
 'neighborhood',
 'postal_code',
 'latitude',
 'longitude',
 'monday',
 'tuesday',
 'wednesday',
 'thursday',
 'friday',
 'saturday',
 'sunday',
 'Acai Bowls',
 'Afghan',
 'African',
 'American (New)',
 'American (Traditional)',
 'Arcades',
 'Asian Fusion',
 'Bagels',
 'Bakeries',
 'Barbeque',
 'Bars',
 'Bed & Breakfast',
 'Beer',
 'Beer Bar',
 'Bistros',
 'Brasseries',
 'Brazilian',
 'Breakfast & Brunch',
 'Breweries',
 'British',
 'Bubble Tea',
 'Buffets',
 'Burgers',
 'Butcher',
 'Cafes',
 'Cajun/Creole',
 'Canadian (New)',
 'Candy Stores

In [13]:
# subset business columns and reorder
business_features = [
 'stars',
 'price',
 'review_count',
 '1star_ratio',
 '2star_ratio',
 '3star_ratio',
 '4star_ratio',
 '5star_ratio',
 'word_count',
 'text_friendly',
 'text_reasonable',
 'text_local',
 'text_comfortable',
 'text_easy',
 'text_quick',
 'cat_Restaurants',
 'cat_Food',
 'cat_Shopping',
 'cat_Active Life',
 'cat_Arts & Entertainment',
 'cat_Nightlife',
 'cat_Hotels & Travel',
 'cat_Tours',
 'Acai Bowls',
 'Afghan',
 'African',
 'American (New)',
 'American (Traditional)',
 'Arcades',
 'Asian Fusion',
 'Bagels',
 'Bakeries',
 'Barbeque',
 'Bars',
 'Bed & Breakfast',
 'Beer',
 'Beer Bar',
 'Bistros',
 'Brasseries',
 'Brazilian',
 'Breakfast & Brunch',
 'Breweries',
 'British',
 'Bubble Tea',
 'Buffets',
 'Burgers',
 'Butcher',
 'Cafes',
 'Cajun/Creole',
 'Canadian (New)',
 'Candy Stores',
 'Cantonese',
 'Caribbean',
 'Casinos',
 'Caterers',
 'Cheesesteaks',
 'Chicken Shop',
 'Chicken Wings',
 'Chinese',
 'Chocolatiers & Shops',
 'Cocktail Bars',
 'Coffee & Tea',
 'Coffee Roasteries',
 'Comfort Food',
 'Convenience Stores',
 'Creperies',
 'Cupcakes',
 'Custom Cakes',
 'Dance Clubs',
 'Delicatessen',
 'Delis',
 'Department Stores',
 'Desserts',
 'Dim Sum',
 'Diners',
 'Dive Bars',
 'Do-It-Yourself Food',
 'Donairs',
 'Donuts',
 'Drugstores',
 'Ethiopian',
 'Ethnic Food',
 'Event Planning & Services',
 'Falafel',
 'Farmers Market',
 'Fashion',
 'Fast Food',
 'Filipino',
 'Fish & Chips',
 'Flowers & Gifts',
 'Food Court',
 'Food Delivery Services',
 'Food Stands',
 'Food Trucks',
 'French',
 'Fruits & Veggies',
 'Gastropubs',
 'Gelato',
 'German',
 'Gluten-Free',
 'Golf',
 'Greek',
 'Grocery',
 'Halal',
 'Hawaiian',
 'Health Markets',
 'Home & Garden',
 'Hookah Bars',
 'Hot Dogs',
 'Hot Pot',
 'Hotels',
 'Ice Cream & Frozen Yogurt',
 'Imported Food',
 'Indian',
 'International Grocery',
 'Internet Cafes',
 'Irish',
 'Irish Pub',
 'Italian',
 'Japanese',
 'Juice Bars & Smoothies',
 'Karaoke',
 'Kebab',
 'Korean',
 'Kosher',
 'Latin American',
 'Lebanese',
 'Local Flavor',
 'Lounges',
 'Meat Shops',
 'Mediterranean',
 'Mexican',
 'Middle Eastern',
 'Modern European',
 'Music Venues',
 'Noodles',
 'Organic Stores',
 'Pakistani',
 'Pan Asian',
 'Party & Event Planning',
 'Pasta Shops',
 'Patisserie/Cake Shop',
 'Persian/Iranian',
 'Peruvian',
 'Pizza',
 'Poke',
 'Polish',
 'Pool Halls',
 'Portuguese',
 'Poutineries',
 'Pubs',
 'Ramen',
 'Salad',
 'Salvadoran',
 'Sandwiches',
 'Seafood',
 'Seafood Markets',
 'Shaved Ice',
 'Shopping Centers',
 'Soul Food',
 'Soup',
 'Southern',
 'Spanish',
 'Specialty Food',
 'Sports Bars',
 'Steakhouses',
 'Street Vendors',
 'Sushi Bars',
 'Szechuan',
 'Tacos',
 'Taiwanese',
 'Tapas Bars',
 'Tapas/Small Plates',
 'Tea Rooms',
 'Tex-Mex',
 'Thai',
 'Turkish',
 'Vegan',
 'Vegetarian',
 'Venues & Event Spaces',
 'Vietnamese',
 'Waffles',
 'Wine & Spirits',
 'Wine Bars',
 'Wraps'
]

# separate out excluded columns
business_features_detail = [
    'name',
    'city',
    'las_vegas',
    'state',
    'address',
    'neighborhood',
    'postal_code',
    'latitude',
    'longitude',
    'monday',
    'tuesday',
    'wednesday',
    'thursday',
    'friday',
    'saturday',
    'sunday',
    'text'
]

In [14]:
# subset user columns and reorder
user_features = [
 'review_count',
 'average_stars',
 'yelping_since',
 'friend_count'
 'cool',
 'funny',
 'useful'
                ]

#### Building the ID mappings

In [37]:
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit(r['user_id'], r['business_id'])
print(type(dataset))

no_users, no_items = dataset.interactions_shape()
print('Number of users: {}\nNumber of businesses: {}'.format(no_users, no_items))

<class 'lightfm.data.Dataset'>
Number of users: 482806
Number of businesses: 8329


#### Building the interactions matrix

In [39]:
(interactions, weights) = dataset.build_interactions([(x['user_id'],
                                                      x['business_id'],
                                                      x['sentiment']) for ind, x in r.iterrows()])

print(repr(interactions))

<482806x8329 sparse matrix of type '<class 'numpy.int32'>'
	with 1291335 stored elements in COOrdinate format>


#### Add business and user features

In [74]:
dataset.fit_partial(items=b.index, item_features=business_features)
#dataset.fit_partial(users=u.index, user_features=user_features)

In [75]:
# check data
print(dataset.model_dimensions())
print(dataset.item_features_shape())
print(dataset.user_features_shape())
print(dataset.interactions_shape())

(482812, 8514)
(8329, 8514)
(482806, 482812)
(482806, 8329)


#### Build business and user features

In [76]:
item_features = dataset.build_item_features((index, business_features) for index, _x in b.iterrows())
#user_features = dataset.build_user_features((index, user_features) for index, _x in u.iterrows())

print(repr(item_features))
#print(repr(user_features))

<8329x8514 sparse matrix of type '<class 'numpy.float32'>'
	with 1549194 stored elements in Compressed Sparse Row format>


#### Building a model

In [77]:
# train-test split
seed = 42
train, test = random_train_test_split(interactions,
                                      test_percentage=0.2,
                                      random_state=np.random.RandomState(seed))

print(f'The dataset has {train.shape[0]} users and {train.shape[1]} items, '
      'with {train.getnnz()} interactions in the train and {test.getnnz() interactions in the test set.')

train.multiply(test).nnz == 0 # make sure train and test are truly disjoint

The dataset has 482806 users and 8329 items, with {train.getnnz()} interactions in the train and {test.getnnz() interactions in the test set.


True

#### Compare CF and Hybrid

In [79]:
model_cf_logitstic = LightFM(no_components=50, loss='logistic')
model_cf_logitstic.fit(train, epochs=5, num_threads=2)

%time print(f'CF_warp train AUC: {auc_score(model_cf_logitstic, train, num_threads=2).mean():.4f}')
%time print(f'CF_warp test AUC: {auc_score(model_cf_logitstic, test, num_threads=2).mean():.4f}')
%time print(f'CF_warp train precision: {precision_at_k(model_cf_logitstic, train, num_threads=2).mean():.4f}')
%time print(f'CF_warp test precision: {precision_at_k(model_cf_logitstic, test, num_threads=2).mean():.4f}')

CF_warp train AUC: 0.8596
Wall time: 8min 38s
CF_warp test AUC: 0.8567
Wall time: 3min 21s
CF_warp train precision: 0.0100
Wall time: 9min 7s
CF_warp test precision: 0.0066
Wall time: 3min 28s


In [80]:
model_cf_warp = LightFM(no_components=50, loss='warp')
model_cf_warp.fit(train, epochs=5, num_threads=2)

%time print(f'CF_warp train AUC: {auc_score(model_cf_warp, train, num_threads=2).mean():.4f}')
%time print(f'CF_warp test AUC: {auc_score(model_cf_warp, test, num_threads=2).mean():.4f}')
%time print(f'CF_warp train precision: {precision_at_k(model_cf_warp, train, num_threads=2).mean():.4f}')
%time print(f'CF_warp test precision: {precision_at_k(model_cf_warp, test, num_threads=2).mean():.4f}')

CF_warp train AUC: 0.9897
Wall time: 9min 22s
CF_warp test AUC: 0.8910
Wall time: 3min 36s
CF_warp train precision: 0.0970
Wall time: 13min 34s
CF_warp test precision: 0.0091
Wall time: 3min 35s


In [81]:
model_hybrid_logitstic = LightFM(no_components=50, loss='logistic')
model_hybrid_logitstic.fit(train, item_features=item_features.T, epochs=5, num_threads=2)

%time print(f'CF_warp train AUC: {auc_score(model_hybrid_logitstic, train, num_threads=2).mean():.4f}')
%time print(f'CF_warp test AUC: {auc_score(model_hybrid_logitstic, test, num_threads=2).mean():.4f}')
%time print(f'CF_warp train precision: {precision_at_k(model_hybrid_logitstic, train, num_threads=2).mean():.4f}')
%time print(f'CF_warp test precision: {precision_at_k(model_hybrid_logitstic, test, num_threads=2).mean():.4f}')

CF_warp train AUC: 0.8598
Wall time: 11min 56s
CF_warp test AUC: 0.8568
Wall time: 5min 55s
CF_warp train precision: 0.0100
Wall time: 9min 19s
CF_warp test precision: 0.0066
Wall time: 3min 29s


In [82]:
model_hybrid_warp = LightFM(no_components=50, loss='warp')
model_hybrid_warp.fit(train, item_features=item_features.T, epochs=5, num_threads=2)

%time print(f'CF_warp train AUC: {auc_score(model_hybrid_warp, train, num_threads=2).mean():.4f}')
%time print(f'CF_warp test AUC: {auc_score(model_hybrid_warp, test, num_threads=2).mean():.4f}')
%time print(f'CF_warp train precision: {precision_at_k(model_hybrid_warp, train, num_threads=2).mean():.4f}')
%time print(f'CF_warp test precision: {precision_at_k(model_hybrid_warp, test, num_threads=2).mean():.4f}')

CF_warp train AUC: 0.8583
Wall time: 9min 2s
CF_warp test AUC: 0.8559
Wall time: 3min 23s
CF_warp train precision: 0.0100
Wall time: 9min 8s
CF_warp test precision: 0.0066
Wall time: 3min 30s


In [91]:
user_id_mapping, user_feature_mapping, item_id_mapping, item_feature_mapping = dataset.mapping()

In [249]:
# columns to return for recommendation output
business_features_reco = [
    'name',
    'stars',
    'price',
    'postal_code',
    'latitude',
    'longitude',
    'monday',
    'tuesday',
    'wednesday',
    'thursday',
    'friday',
    'saturday',
    'sunday'
]

In [326]:
def make_recommendation(model, user_ids, city='Las Vegas', top_n=3):
    """
    """
    business_map = {i: item_id_mapping.get(i) for i in b.index[b.city == city]}
    
    for user_id in user_ids:
        known_positives = list(r.business_id[(r.user_id == user_id) & (r.city == city) & (r.sentiment == 1)])
        
        user_map = user_id_mapping.get(user_id)
        
        scores = model.predict(user_ids=user_map, item_ids=list(business_map.values()), num_threads=2)
        recommendation = dict(zip(list(business_map.keys()), scores))
        top_items = dict(sorted(recommendation.items(), key=lambda x: x[1], reverse=True))

        print(f"User {user_id}")
        print("     Known positives:")
        print(b[["name", "stars", "price"]][b.index.isin(known_positives[:top_n])])

        print("     Recommended:")
        print(b[["name", "stars", "price"]][b.index.isin(top_items.keys()[:top_n])])
        
        return recommendation

In [329]:
list(r.business_id[(r.user_id == '-hJ9Q0OSYqdOTVBLOMdg7A') & (r.city == "Las Vegas") & (r.sentiment == 1)])

['NG2mL-wQQfBdsXsyUbw8pA',
 'pomGBqfbxcqPv14c3XH-ZQ',
 'qJEtXdrOQeo9py0Wqr6LKQ',
 'UDE4VL0k8Zrko7RkJ4agGQ',
 'FNe5PPA9pyj8FjcDefCBpg',
 '2iTsRqUsPGRH1li1WVRvKQ']

In [327]:
make_recommendation(model_cf_logitstic, ['-hJ9Q0OSYqdOTVBLOMdg7A'], city='Las Vegas', top_n=20)

User -hJ9Q0OSYqdOTVBLOMdg7A
     Known positives:
                                             name  stars  price
business_id                                                    
2iTsRqUsPGRH1li1WVRvKQ             Carson Kitchen    4.5    2.0
FNe5PPA9pyj8FjcDefCBpg                 Weera Thai    4.5    2.0
NG2mL-wQQfBdsXsyUbw8pA  Capriotti's Sandwich Shop    3.0    2.0
UDE4VL0k8Zrko7RkJ4agGQ            Little Dumpling    3.5    1.0
pomGBqfbxcqPv14c3XH-ZQ   Leticia's Mexican Cocina    4.0    2.0
qJEtXdrOQeo9py0Wqr6LKQ     Blaze Fast Fired Pizza    4.0    1.0
     Recommended:


TypeError: 'dict_keys' object is not subscriptable

In [262]:
reco_ex_cf = make_recommendation(model_cf_logitstic, ['-hJ9Q0OSYqdOTVBLOMdg7A'], city='Las Vegas', top_n=20)

User -hJ9Q0OSYqdOTVBLOMdg7A
     Known positives:
                                             name  stars  price
business_id                                                    
2iTsRqUsPGRH1li1WVRvKQ             Carson Kitchen    4.5    2.0
FNe5PPA9pyj8FjcDefCBpg                 Weera Thai    4.5    2.0
NG2mL-wQQfBdsXsyUbw8pA  Capriotti's Sandwich Shop    3.0    2.0
UDE4VL0k8Zrko7RkJ4agGQ            Little Dumpling    3.5    1.0
pomGBqfbxcqPv14c3XH-ZQ   Leticia's Mexican Cocina    4.0    2.0
qJEtXdrOQeo9py0Wqr6LKQ     Blaze Fast Fired Pizza    4.0    1.0
     Recommended:
                                        name  stars  price
business_id                                               
-zbJp3NGEhvmNOiFziN8Lg      Lopez Restaurant    4.5    2.0
-zkn2p1mu9g-kbGNdM1lUA          Chinos Locos    4.0    1.0
0zAJMAk4TbSm9XNQr9ks0A                Subway    3.0    1.0
0zGeYN8ReYwVj27U7EHOJA         American Cafe    2.5    2.0
0zoXYHq82haayMaV952jEQ   Greens and Proteins    4.0    2.0
2zSr7jN

In [263]:
reco_ex_hybrid = make_recommendation(model_hybrid_logitstic, ['-hJ9Q0OSYqdOTVBLOMdg7A'], city='Las Vegas', top_n=20)

User -hJ9Q0OSYqdOTVBLOMdg7A
     Known positives:
                                             name  stars  price
business_id                                                    
2iTsRqUsPGRH1li1WVRvKQ             Carson Kitchen    4.5    2.0
FNe5PPA9pyj8FjcDefCBpg                 Weera Thai    4.5    2.0
NG2mL-wQQfBdsXsyUbw8pA  Capriotti's Sandwich Shop    3.0    2.0
UDE4VL0k8Zrko7RkJ4agGQ            Little Dumpling    3.5    1.0
pomGBqfbxcqPv14c3XH-ZQ   Leticia's Mexican Cocina    4.0    2.0
qJEtXdrOQeo9py0Wqr6LKQ     Blaze Fast Fired Pizza    4.0    1.0
     Recommended:
                                        name  stars  price
business_id                                               
-zbJp3NGEhvmNOiFziN8Lg      Lopez Restaurant    4.5    2.0
-zkn2p1mu9g-kbGNdM1lUA          Chinos Locos    4.0    1.0
0zAJMAk4TbSm9XNQr9ks0A                Subway    3.0    1.0
0zGeYN8ReYwVj27U7EHOJA         American Cafe    2.5    2.0
0zoXYHq82haayMaV952jEQ   Greens and Proteins    4.0    2.0
2zSr7jN

In [330]:
len(reco_ex_hybrid)

5077

In [316]:
reco_ex_hybrid

{'--9e1ONYQuAa-CB_Rrw7Tw': 1.3800417184829712,
 '-0BxAGlIk5DJAGVkpqBXxg': 0.6563155651092529,
 '-1m9o3vGRA8IBPNvNqKLmA': 0.7207193374633789,
 '-1uVyyy3UXNA9WoPucNniA': 0.6517390012741089,
 '-3zffZUHoY8bQjGfPSoBKQ': 0.9303737878799438,
 '-46pFijv3f2jZH-_Ze6CQA': 0.6474716663360596,
 '-8R_-EkGpUhBk55K9Dd4mg': 0.6906703114509583,
 '-8ZiMXZReeTD3kwEvS0Lww': 0.6971354484558105,
 '-95mbLJsa0CxXhpaNL4LvA': 0.9248393774032593,
 '-9Ir5OiFiOszIgeFgalXcw': 0.6544783711433411,
 '-9YyInW1wapzdNZrhQJ9dg': 0.7305017709732056,
 '-AD5PiuJHgdUcAK-Vxao2A': 0.7632898688316345,
 '-ADtl9bLp8wNqYX1k3KuxA': 0.8152552247047424,
 '-Bf8BQ3yMk8U2f45r2DRKw': 0.7164429426193237,
 '-Bv-HHUs8aHzDrdWcZHn8w': 0.674694836139679,
 '-CQokjildrY7UZezXCdEBw': 0.6547957062721252,
 '-EohS3nXZGl6odE9Lx5BSw': 0.6420024633407593,
 '-FNquqGseSCVMWo7KbK-Tg': 0.6914891004562378,
 '-FcZY7a7qgxTUlTvwuyJnQ': 0.7454558610916138,
 '-G7MPSNBpxRJmtrJxdwt7A': 0.6560462713241577,
 '-G9DFmuokdYk_H35ezRkAw': 0.6417888402938843,
 '-INdswGdyRyy

In [325]:
dict(sorted(reco_ex_hybrid.items(), key=lambda x: x[1], reverse=True))

{'4JNXUYY8wbaaDmk3BPzlWw': 4.313278675079346,
 'RESDUcs7fIiihp38-d6_6g': 4.307117462158203,
 'K7lWdNUhCbcnEvI0NhGewg': 3.6470019817352295,
 'f4x1YBxkLrZg652xt2KR5g': 3.2104856967926025,
 'DkYS3arLOhA8si5uUEmHOw': 2.9956014156341553,
 '2weQS-RnoOBhb1KsHKyoSQ': 2.6513471603393555,
 'ujHiaprwCQ5ewziu0Vi9rw': 2.573176622390747,
 'iCQpiavjjPzJ5_3gPD5Ebg': 2.5350687503814697,
 'hihud--QRriCYZw1zZvW4g': 2.4925694465637207,
 'El4FC8jcawUVgw_0EIcbaQ': 2.4409518241882324,
 'rcaPajgKOJC2vo_l3xa42A': 2.419630289077759,
 'OETh78qcgDltvHULowwhJg': 2.2092912197113037,
 'XZbuPXdyA0ZtTu3AzqtQhg': 2.142956495285034,
 '7sPNbCx7vGAaH7SbNPZ6oA': 2.1377856731414795,
 'g8OnV26ywJlZpezdBnOWUQ': 2.118083953857422,
 'HhVmDybpU7L50Kb5A0jXTg': 2.0949158668518066,
 'XXW_OFaYQkkGOGniujZFHg': 2.085341215133667,
 'P7pxQFqr7yBKMMI2J51udw': 2.0615015029907227,
 'YJ8ljUhLsz6CtT_2ORNFmg': 2.059823513031006,
 'RwMLuOkImBIqqYj4SSKSPg': 2.0507020950317383,
 '3kdSl5mo9dWC4clrQjEDGg': 2.045391082763672,
 'faPVqws-x-5k2CQKDNtH

In [264]:
# append recommendation score to the original matrix
b_example = b[business_features_reco][b["city"] == "Las Vegas"]
b_example["score_cf"] = b_example.index.map(reco_ex_cf)
b_example["score_hybrid"] = b_example.index.map(reco_ex_hybrid)

In [320]:
b_example[b_example.index == '-zbJp3NGEhvmNOiFziN8Lg']

,name,categories_clean,score_cf,score_hybrid,stars,price,postal_code,latitude,longitude,monday,tuesday,wednesday,thursday,friday,saturday,sunday
business_id,,,,,,,,,,,,,,,,
-zbJp3NGEhvmNOiFziN8Lg,Lopez Restaurant,['Mexican'],0.558138,0.638243,4.5,2.0,89104,36.158845,-115.125984,0,0,0,0,0,0,0


In [303]:
b2["categories"][0]

'Steakhouses, Restaurants, Cajun/Creole'

In [305]:
b2["categories_clean"][0]

"['Steakhouses', 'Cajun/Creole']"

In [306]:
b_example["categories_clean"] = b_example.index.map(b2["categories_clean"])

In [307]:
# reorder columns
col = ['name',
 'categories_clean',
 'score_cf',
 'score_hybrid',
 'stars',
 'price',
 'postal_code',
 'latitude',
 'longitude',
 'monday',
 'tuesday',
 'wednesday',
 'thursday',
 'friday',
 'saturday',
 'sunday'
      ]
b_example = b_example[col]
b_example.head()

,name,categories_clean,score_cf,score_hybrid,stars,price,postal_code,latitude,longitude,monday,tuesday,wednesday,thursday,friday,saturday,sunday
business_id,,,,,,,,,,,,,,,,
--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"['Steakhouses', 'Cajun/Creole']",4.064405,1.380042,4.0,4.0,89109,36.123183,-115.169190,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0,11:30-14:0
-0BxAGlIk5DJAGVkpqBXxg,The Coffee Bean & Tea Leaf,['Coffee & Tea'],1.340302,0.656316,3.0,2.0,89162,36.122935,-115.168528,0,0,0,0,0,0,0
-1m9o3vGRA8IBPNvNqKLmA,Bavette's Steakhouse & Bar,"['African', 'Bars', 'American (New)', 'Cocktai...",2.189195,0.720719,4.5,4.0,89109,36.104330,-115.175593,17:0-22:0,17:0-22:0,17:0-22:0,17:0-22:0,17:0-23:0,17:0-23:0,17:0-22:0
-1uVyyy3UXNA9WoPucNniA,La Tapatia Market,"['Meat Shops', 'Grocery', 'Seafood Markets', '...",1.210479,0.651739,5.0,1.0,89123,36.029110,-115.116823,6:0-20:0,6:0-20:0,6:0-20:0,6:0-20:0,6:0-20:0,6:0-20:0,6:0-20:0
-3zffZUHoY8bQjGfPSoBKQ,Michael Mina,"['American (New)', 'Bars', 'Seafood']",3.212997,0.930374,4.0,4.0,89109,36.111990,-115.177001,17:0-22:0,17:0-22:0,17:0-22:0,17:0-22:0,17:0-22:0,17:0-22:0,17:30-22:0


In [309]:
b_example.score_hybrid.describe()

count    5077.000000
mean        0.723847
std         0.188613
min         0.637031
25%         0.645440
50%         0.663584
75%         0.725208
max         4.313279
Name: score_hybrid, dtype: float64

In [308]:
# save the example output

b_example.to_csv("recommendation_-hJ9Q0OSYqdOTVBLOMdg7A.csv")

In [226]:
r[r["user_id"] == '-hJ9Q0OSYqdOTVBLOMdg7A']

,business_id,cool,date,funny,stars,text,useful,user_id,sentiment,city
review_id,,,,,,,,,,
08qJLkJdAVY6L_3R0Jqffw,Tp07u-0AOFRfQhtNpAXaQA,0,2007-02-04,0,4,I really like this place.\n\nThe people workin...,0,-hJ9Q0OSYqdOTVBLOMdg7A,1,Phoenix
DcLdUnDi5ATXvywe4yJMXw,pSQFynH1VxkfSmehRXlZWw,3,2007-02-04,2,2,"I saw the mention in that book-- was it ""The B...",4,-hJ9Q0OSYqdOTVBLOMdg7A,-1,Phoenix
BoguEYl3yC1bVU0KjdRqIw,NG2mL-wQQfBdsXsyUbw8pA,0,2018-01-19,0,4,Update:\n\nI'm adding 1 star because today I a...,0,-hJ9Q0OSYqdOTVBLOMdg7A,1,Las Vegas
o8F3m8mQiuWYTbJEmFIyyw,pomGBqfbxcqPv14c3XH-ZQ,0,2016-10-07,3,5,I was here once a long time ago and then again...,0,-hJ9Q0OSYqdOTVBLOMdg7A,1,Las Vegas
e96mj8MSNvJ3OWoA_90x9Q,Lau3N654yNyWdIIybea1uA,0,2007-05-13,0,3,Went on a lark after a few days at the convent...,1,-hJ9Q0OSYqdOTVBLOMdg7A,-1,Las Vegas
fu0nJ48BBtuGbitGgnoJOw,C3gKBIsZKR3bp43BbOu8qA,0,2007-02-04,0,4,I like Thai Rama.\n\nI was tempted to give it ...,0,-hJ9Q0OSYqdOTVBLOMdg7A,1,Phoenix
gGtN0IzeSh-BBmIcRNMJfQ,qJEtXdrOQeo9py0Wqr6LKQ,0,2018-03-10,0,5,"I am skeptical of pizza chains, but Blaze (the...",0,-hJ9Q0OSYqdOTVBLOMdg7A,1,Las Vegas
eE6KFM3a8IoKRf8bK9Liyw,079CV1EE5WLdQqVEVYFeHQ,0,2007-02-13,0,4,"I really, really like this place a lot.\n\nThi...",3,-hJ9Q0OSYqdOTVBLOMdg7A,1,Phoenix
sQgkkfr72x2jfxK5BNMWng,yz3JQk5Lqbin8mtNfDcy_w,0,2007-02-06,0,4,Easily the best part of waiting around for a d...,1,-hJ9Q0OSYqdOTVBLOMdg7A,1,Phoenix


In [289]:
def rec_accuracy(model, users, city='Las Vegas', multiplier=2):
    """
    1. takes in list of user_id
    2. keep known positives and its length
    3. return recommendations by known postivie length x N
    4. count number of known positives in the recommendation list
    5. calculate true positive ratio
    """
    accuracy = []
    
    business_map = {i: item_id_mapping.get(i) for i in b.index[b.city == city]}
        
    for user_id in users:
        known_positives = list(r.business_id[(r.user_id == user_id) & (r.city == city) & (r.sentiment == 1)])
        
        # skip if user has no known_positives
        if len(known_positives) == 0:
            continue

        user_map = user_id_mapping.get(user_id)
        scores = model.predict(user_ids=user_map, item_ids=list(business_map.values()), num_threads=2)
        result = dict(zip(list(business_map.keys()), scores))
        #top_items = dict(filter(lambda x: x[1] > 0, result.items()))
        top_items = sorted(result.keys(), key=lambda x: x[1], reverse=True)[:len(known_positives)*multiplier]
        true_positives = list(set(top_items) & set(known_positives))
        acc = len(true_positives)/len(known_positives)
        accuracy.append(acc)
    
    #print(f"Out of {len(top_items)} recommendations, ", f"{len(true_positives)} out of ", f"{len(known_positives)} made it to the recommendation.")
    print(f"There were total {len(accuracy)} users in the evaluation.")
    
    return np.mean(accuracy)

In [ ]:
scores = model.predict(user_ids=user_map, item_ids=list(business_map.values()), num_threads=2)

In [285]:
rec_accuracy(model_cf_warp, ['-hJ9Q0OSYqdOTVBLOMdg7A'], multiplier=100)

There were total 1 users in the evaluation.


0.8333333333333334

In [216]:
rec_accuracy(model_hybrid_warp, ['-hJ9Q0OSYqdOTVBLOMdg7A'])

0.0

#### Find the test user set

In [267]:
test.

<482806x8329 sparse matrix of type '<class 'numpy.int32'>'
	with 258267 stored elements in COOrdinate format>

In [287]:
user_matrix = u_lv_ph
sample_size = 1000
test_idx = random.sample(range(len(user_matrix)), sample_size)
test_users = [user_matrix[i] for i in test_idx]

In [288]:
rec_accuracy(model_cf_logitstic, test_users, multiplier=10)

There were total 816 users in the evaluation.


1.0

In [290]:
# evaluation with 1000 users
rec_accuracy(model_cf_logitstic, test_users, multiplier=10)

There were total 816 users in the evaluation.


0.005647320528982533

In [291]:
# evaluation with 1000 users
rec_accuracy(model_cf_logitstic, test_users, multiplier=100)

There were total 816 users in the evaluation.


0.05636855495584717

In [331]:
# evaluation with 1000 users
rec_accuracy(model_hybrid_logitstic, test_users, multiplier=100)

There were total 816 users in the evaluation.


0.05636855495584717

In [281]:
# evaluation with 100 users
rec_accuracy(model_cf_logitstic, test_users, multiplier=10)

There were total 81 users in the evaluation.


0.005658436213991769

In [269]:
rec_accuracy(model_hybrid_warp, test_users, multiplier=10)

0.0041843685695392065

In [219]:
rec_accuracy(model_cf_warp, test_users)

0.002380952380952381

In [144]:
known_positives = list(r.business_id[(r.user_id == '-hJ9Q0OSYqdOTVBLOMdg7A') & (r.city == 'Las Vegas') & (r.sentiment == 1)])
known_positives

['NG2mL-wQQfBdsXsyUbw8pA',
 'pomGBqfbxcqPv14c3XH-ZQ',
 'qJEtXdrOQeo9py0Wqr6LKQ',
 'UDE4VL0k8Zrko7RkJ4agGQ',
 'FNe5PPA9pyj8FjcDefCBpg',
 '2iTsRqUsPGRH1li1WVRvKQ']

In [162]:
business_map = {i: item_id_mapping.get(i) for i in b.index[b.city == 'Las Vegas']}
len(business_map)

5077

In [149]:
user_id_mapping.get('-hJ9Q0OSYqdOTVBLOMdg7A')

361498

In [181]:
scores = model_cf_warp.predict(user_ids=361498, item_ids=list(business_map.values()), num_threads=2)
len(scores)

5077

In [182]:
result = dict(zip(list(business_map.keys()), scores))
result

{'--9e1ONYQuAa-CB_Rrw7Tw': -0.6902596950531006,
 '-0BxAGlIk5DJAGVkpqBXxg': -2.9332828521728516,
 '-1m9o3vGRA8IBPNvNqKLmA': -1.1170930862426758,
 '-1uVyyy3UXNA9WoPucNniA': -2.2192018032073975,
 '-3zffZUHoY8bQjGfPSoBKQ': -1.9691944122314453,
 '-46pFijv3f2jZH-_Ze6CQA': -2.6695446968078613,
 '-8R_-EkGpUhBk55K9Dd4mg': -1.5633412599563599,
 '-8ZiMXZReeTD3kwEvS0Lww': -0.5208823084831238,
 '-95mbLJsa0CxXhpaNL4LvA': -1.1259938478469849,
 '-9Ir5OiFiOszIgeFgalXcw': -2.049160957336426,
 '-9YyInW1wapzdNZrhQJ9dg': -0.5131211876869202,
 '-AD5PiuJHgdUcAK-Vxao2A': -1.3903499841690063,
 '-ADtl9bLp8wNqYX1k3KuxA': 0.2531701624393463,
 '-Bf8BQ3yMk8U2f45r2DRKw': -0.382830411195755,
 '-Bv-HHUs8aHzDrdWcZHn8w': -2.3828256130218506,
 '-CQokjildrY7UZezXCdEBw': -2.2089426517486572,
 '-EohS3nXZGl6odE9Lx5BSw': -1.6018083095550537,
 '-FNquqGseSCVMWo7KbK-Tg': -1.6883459091186523,
 '-FcZY7a7qgxTUlTvwuyJnQ': -0.716923177242279,
 '-G7MPSNBpxRJmtrJxdwt7A': -1.7670224905014038,
 '-G9DFmuokdYk_H35ezRkAw': -2.55497670173645

In [184]:
top_items = sorted(result, key=lambda x: x[1], reverse=True)[:len(known_positives)*2]
top_items

['-zbJp3NGEhvmNOiFziN8Lg',
 '-zkn2p1mu9g-kbGNdM1lUA',
 '0zAJMAk4TbSm9XNQr9ks0A',
 '0zGeYN8ReYwVj27U7EHOJA',
 '0zoXYHq82haayMaV952jEQ',
 '2zSr7jNNWpA-IfD74wOlTA',
 '2zufEbSI_D8t3WPCgShw-w',
 '3zi81XrZYIMjaJnC0JLSsg',
 '4zqoILJLZ-3F8VVNr2hg1w',
 '5z1WIr7E9P2CSzyN5seeSA',
 '6zKU06XeCBdRDnzgNB5NmQ',
 '6zuRum92Zg4G__ml6NceKg']